In [1]:
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import json
from pathlib import Path
import csv

In [ ]:
import wandb
wandb.init(project="pair-programming")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yahata. Use `wandb login --relogin` to force relogin


In [2]:
class CFG():
    pretrained_model = '/larch/share/bert/NICT_pretrained_models/NICT_BERT-base_JapaneseWikipedia_32K_BPE'
    pretrained_model_roberta = "nlp-waseda/roberta-base-japanese"
    data_path = '/mnt/hinoki/karai/KUCI'
    max_seq_len = 128
    batch_size = 16
    lr = 2e-5
    weight_decay = 0.01
    seed = 0
    epoch = 3
    warmup_ratio = 0.033
    save_path = "./result/bert"
    save_path_roberta = "./result/roberta"

In [34]:
wandb.log({
    'max_seq_len': CFG.max_seq_len, 
    'batch_size': CFG.batch_size,
    'lr': CFG.lr,
    'weight_decay': CFG.weight_decay,
    'seed': CFG.seed,
    'epoch': CFG.epoch,
    'warmup_ratio': CFG.epoch
    })

In [4]:
save_path = Path(CFG.save_path)
save_path.mkdir(exist_ok=True, parents=True)

In [5]:
save_path = Path(CFG.save_path_roberta)
save_path.mkdir(exist_ok=True, parents=True)

In [5]:
import random
import numpy as np
import torch
import os
import re


def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def set_device(gpuid: str) -> torch.device:
    if gpuid and torch.cuda.is_available():
        assert re.fullmatch(r"[0-7]", gpuid) is not None, "invalid way to specify gpuid"
        os.environ["CUDA_VISIBLE_DEVICES"] = gpuid
        device = torch.device(f"cuda:{gpuid}")
    else:
        device = torch.device("cpu")

    return device

In [6]:
set_seed(CFG.seed)
device = set_device("0")
print(device)

cpu


In [7]:
tokenizer = BertTokenizer.from_pretrained(
    CFG.pretrained_model, do_lower_case=False, do_basic_tokenize=False
)

print(tokenizer.tokenize('今日は曇りです'))
print(tokenizer([['今日は曇りです', '明日は晴れるといいな'],['明日はピクニックです', 'お弁当楽しみだな']]))

['今日', '##は', '##曇', '##り', '##です']
{'input_ids': [[2, 8016, 26343, 29756, 26483, 26287, 3, 15108, 26343, 29746, 26507, 26292, 25974, 26302, 3], [2, 15108, 26343, 28149, 26894, 27927, 26287, 3, 274, 29556, 29573, 29822, 26149, 26222, 26302, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [8]:
class PairPro(Dataset):
    def __init__(self, path, tokenizer, max_seq_len, is_test=False):
        self.is_test = is_test
        self.label2int = {'a': 0, 'b': 1, 'c': 2, 'd': 3}
        self.labels, self.contexts, self.choices = self.load(path)
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    
    def load(self, path):
        label_list = []
        context_list = []
        choice_list = []
        with open(path) as f:
            for i, line in enumerate(f):
                problem = json.loads(line)
                if self.is_test:
                    label_list.append(-1)
                else:
                    label_list.append(self.label2int[problem['label']])
                context_list.append(problem['context'])
                choice_list.append([problem['choice_a'],problem['choice_b'],problem['choice_c'],problem['choice_d']])
        assert len(label_list) == len(context_list), "長さが違います"
        return label_list, context_list, choice_list

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        input_list = []
        for choice in self.choices[idx]:
            input_list.append([self.contexts[idx], choice])    
        return self.labels[idx], self.tokenizer(
            input_list,
            max_length=self.max_seq_len, 
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            )

In [6]:
traindataset = PairPro(CFG.data_path+'/train.jsonl', tokenizer, CFG.max_seq_len)
print(traindataset[0])  # pairの要素を参照すると__getitem__が呼び出される
devdataset = PairPro(CFG.data_path+'/development.jsonl', tokenizer, CFG.max_seq_len)
testdataset = PairPro(CFG.data_path+'/test.jsonl', tokenizer, CFG.max_seq_len, is_test=True)

NameError: name 'PairPro' is not defined

In [9]:
traindataloader = DataLoader(traindataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2)
devdataloader = DataLoader(devdataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2)
testdataloader = DataLoader(testdataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

In [10]:
class BERTPairPro(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            pretrained_model, output_attentions=False
        )
        self.linear = nn.Linear(
            self.bert.config.hidden_size, 1
        )
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        batch_size, n_choice, seq_len = input_ids.shape     # 16,4,128
        input_ids = input_ids.view(batch_size*n_choice, seq_len)        # 64,128
        attention_mask = attention_mask.view(batch_size*n_choice, seq_len)
        token_type_ids = token_type_ids.view(batch_size*n_choice, seq_len)
        # print(input_ids.shape)
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls = output['pooler_output']   # 64, 768
        output = self.linear(cls)   # 64, 1
        output = output.squeeze(1).view(batch_size, n_choice)
        return output

In [12]:
model = BERTPairPro(CFG.pretrained_model)
model = model.to(device)

Some weights of the model checkpoint at /larch/share/bert/NICT_pretrained_models/NICT_BERT-base_JapaneseWikipedia_32K_BPE were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
model = BERTPairPro(CFG.pretrained_model_roberta)
model = model.to(device)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


In [12]:
# for i, batch in enumerate(PairProdataloader):
#     #print(batch)
#     label, batch = batch
#     if i == 0:
#         output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#         print(output)
#         break

In [12]:
optimizer = AdamW(
    model.parameters(),
    lr=CFG.lr,
    weight_decay=CFG.weight_decay,
    no_deprecation_warning=True
)

In [13]:
num_training_steps = len(traindataloader) * CFG.epoch
num_warmup_steps = num_training_steps * CFG.warmup_ratio
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_training_steps=num_training_steps,
    num_warmup_steps=num_warmup_steps
)

ce_loss = nn.CrossEntropyLoss()

In [15]:
best_score = None
for epoch in range(CFG.epoch):
    total_loss = 0
    model.train()
    train_bar = tqdm(traindataloader)
    for batch_idx, batch in enumerate(train_bar):
        label, batch = batch 
        batch_size = len(batch['input_ids'])
        label = label.to(device)
        batch = {key: value.to(device) for key, value in batch.items()} 
        output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        loss = ce_loss(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * batch_size
        train_bar.set_postfix(
            {
                'lr': scheduler.get_last_lr()[0],
                'loss': round(total_loss / (batch_idx + 1), 3)
            }
        )


    model.eval()
    with torch.no_grad():
        dev_bar = tqdm(devdataloader)
        num_correct = 0
        total_loss = 0
        size = 0 
        for batch_idx, batch in enumerate(dev_bar):
            label, batch = batch 
            batch_size = len(batch['input_ids'])
            label = label.to(device)
            batch = {key: value.to(device) for key, value in batch.items()} 
            output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
            loss = ce_loss(output, label)
            predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
            num_correct += torch.sum(predictions == label).item()
            size += batch_size
            total_loss += loss.item() * batch_size
            score = round(num_correct/size, 3)

            dev_bar.set_postfix(
               {
                'size': size,
                'accuracy': score,
                'loss': round(total_loss / (batch_idx + 1), 3)
                }
            )
            wandb.log(
                {
                    'step': size,
                    'accuracy': score,
                    'loss': total_loss / (batch_idx + 1)
                }
            )


    score = round(num_correct/size, 3)
    if best_score is None or score > best_score:
        torch.save(model.state_dict(), CFG.save_path+"/Checkpoint_best.pth")
        best_score = score



  1%|          | 33/5196 [00:15<39:51,  2.16it/s, lr=1.28e-6, loss=21.3] 

In [36]:
state_dict = torch.load(CFG.save_path+"/Checkpoint_best.pth", map_location=device)
model.load_state_dict(state_dict)   

<All keys matched successfully>

In [14]:
state_dict = torch.load(CFG.save_path_roberta+"/Checkpoint_best.pth", map_location=device)
model.load_state_dict(state_dict)  

<All keys matched successfully>

In [ ]:
prediction = []
model.eval()
with torch.no_grad():
    test_bar = tqdm(testdataloader)
    num_correct = 0
    total_loss = 0
    size = 0 
    for batch_idx, batch in enumerate(test_bar):
        label, batch = batch 
        batch_size = len(batch['input_ids'])
        # label = label.to(device)
        batch = {key: value.to(device) for key, value in batch.items()} 
        output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        # loss = ce_loss(output, label)
        predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
        size += batch_size
        #total_loss += loss.item() * batch_size
        prediction += predictions.tolist()
    
    int2label = {0: 'a', 1: 'b', 2: 'c', 3: 'd'}
    prediction = [int2label[pre] for pre in prediction]
    print(prediction)

    #     #dev_bar.set_postfix(
    #     {
    #         'size': size,
    #         #'accuracy': round(num_correct/size, 3),
    #         #'loss': round(total_loss / (batch_idx + 1), 3)
    #     }
    # )

    

100%|██████████| 32/32 [00:05<00:00,  6.39it/s]

['d', 'b', 'c', 'c', 'a', 'c', 'c', 'b', 'd', 'c', 'c', 'c', 'a', 'b', 'a', 'c', 'd', 'c', 'd', 'b', 'a', 'b', 'b', 'd', 'b', 'a', 'a', 'b', 'a', 'c', 'c', 'd', 'c', 'c', 'c', 'b', 'd', 'a', 'c', 'a', 'd', 'a', 'c', 'd', 'a', 'a', 'c', 'a', 'd', 'a', 'c', 'a', 'a', 'b', 'd', 'c', 'a', 'b', 'b', 'd', 'd', 'c', 'a', 'a', 'a', 'c', 'a', 'd', 'd', 'a', 'a', 'd', 'b', 'b', 'd', 'c', 'd', 'd', 'a', 'c', 'a', 'c', 'c', 'c', 'b', 'c', 'a', 'c', 'a', 'a', 'a', 'c', 'c', 'a', 'c', 'c', 'a', 'b', 'a', 'a', 'd', 'b', 'a', 'c', 'b', 'a', 'a', 'd', 'd', 'b', 'b', 'd', 'b', 'd', 'd', 'd', 'c', 'a', 'd', 'd', 'd', 'a', 'c', 'c', 'a', 'd', 'a', 'b', 'b', 'a', 'b', 'a', 'a', 'a', 'd', 'd', 'd', 'a', 'd', 'b', 'b', 'd', 'c', 'c', 'c', 'a', 'a', 'c', 'b', 'b', 'a', 'b', 'c', 'a', 'd', 'c', 'b', 'b', 'c', 'b', 'b', 'c', 'b', 'c', 'b', 'c', 'd', 'c', 'c', 'c', 'b', 'd', 'a', 'c', 'b', 'c', 'd', 'a', 'c', 'd', 'a', 'b', 'd', 'a', 'c', 'a', 'b', 'd', 'c', 'b', 'b', 'c', 'c', 'b', 'a', 'a', 'c', 'c', 'b', 'b',

In [ ]:
prediction = []
model.eval()
with torch.no_grad():
    test_bar = tqdm(devdataloader)
    num_correct = 0
    total_loss = 0
    size = 0 
    for batch_idx, batch in enumerate(test_bar):
        label, batch = batch 
        batch_size = len(batch['input_ids'])
        # label = label.to(device)
        batch = {key: value.to(device) for key, value in batch.items()} 
        output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        # loss = ce_loss(output, label)
        predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
        size += batch_size
        #total_loss += loss.item() * batch_size
        prediction += predictions.tolist()
    
    int2label = {0: 'a', 1: 'b', 2: 'c', 3: 'd'}
    prediction = [int2label[pre] for pre in prediction]
    print(prediction)

In [ ]:
with open(CFG.save_path+"/test_prediction.csv", "w") as f:
  csv.writer(f).writerows(prediction)
f.close()

# 宿題
- 猿に勝つ

In [3]:
def load(path):
    label_list = []
    context_list = []
    choice_list = []
    agreement_list = []
    with open(path) as f:
        for i, line in enumerate(f):
            problem = json.loads(line)
            label_list.append(problem['label'])
            context_list.append(problem['context'])
            choice_list.append([problem['choice_a'],problem['choice_b'],problem['choice_c'],problem['choice_d']])
            agreement_list.append(problem['agreement'])
    assert len(label_list) == len(context_list), "長さが違います"
    return label_list, context_list, choice_list, agreement_list

In [9]:
predictions_bert = []
with open(CFG.save_path+'/dev_prediction.csv', 'r') as f:
    predict = csv.reader(f)
    for line in predict:
            predictions_bert += line
f.close()
predictions_roberta = []
with open(CFG.save_path_roberta+'/dev_prediction.csv', 'r') as f:
    predict = csv.reader(f)
    for line in predict:
            predictions_roberta += line
f.close()

label, context, choice, agreement = load(CFG.data_path+'/development.jsonl')

false = []
for i, l in enumerate(zip(zip(agreement, label, predictions_bert, predictions_roberta), context, choice)):
    if not l[0][3] == l[0][2]:
        false.append(l)

In [10]:
import pprint as pp
pp.pprint(false[:50])

[((3, 'b', 'c', 'b'),
  '夕方 から は 、 ちょっと だけ 雨 が 降った ので',
  ['今朝 は ５ 時 前 に 目 が 覚め ます',
   '結構 気温 が 下がり ます',
   '日中 、 少し 雪 が とけた みたいだ',
   '熱 が 出る']),
 ((3, 'c', 'a', 'd'),
  '人 の 目 が なければ',
  ['アメリカ みたいな 訳 に は 行か ない',
   'やる 気 が 出 ない',
   '道路 ルール を 守ら ない',
   '巣 に 近づか ない ように して ください']),
 ((3, 'b', 'd', 'b'),
  '他 の ＳＰＦ ５０ の 商品 と 比べる と',
  ['安心 して ご 利用 に なれる', 'つけ 心地 は とっても 軽い', 'すっげー 気 が 楽だ', '使い 勝手 が 悪い']),
 ((2, 'a', 'c', 'a'),
  'お 昼 は お出かけ した ので 、',
  ['出先 の カフェ で パン を あげ ます',
   '祝日 の 堂島 近辺 は 駅前 ビル まで 遠征 する',
   '軽く 朝食 を 済ませる',
   'なかなか 登山 電車 ウォッチング に 行け なく なって しまい ます']),
 ((3, 'a', 'c', 'a'),
  '仕事 内容 が 変わる こと が あれば 、',
  ['多分 もっと ネット 活用 する ように なる',
   'お 車 できて も 便利だ',
   'サービス 内容 など も 異なり ます',
   '逆に 信用 を 失う']),
 ((2, 'd', 'a', 'd'),
  'レベル も 上がった ので',
  ['久々に ウォーキング に 出かけ ます',
   'れいな の 指差す 先 に 視線 を 向ける',
   '明日 は 幼稚園 に 行ける',
   '玉座 方面 へ 向かう']),
 ((3, 'b', 'a', 'b'),
  '遠い 家 で 雨戸 を 開ける 音 が 響く と 、',
  ['なにやら 部屋 の 様子 が おかしい',
   '私 は モヤモヤ した 頭 の まま 起き上がる',
   '緊急 時 の 消火 器 の 使用 が 可能

In [9]:
false = []
for i, predict in enumerate(predictions):
    if not predict == label[i]:
        f = [agreement[i], label[i], predict, context[i], choice[i]]
        false.append(f)
    
agreement_2 = [line for line in false if line[0]==2]

pp.pprint(agreement_2[:20], width=200)

[[2, 'b', 'd', 'ＡＳＰ も 電子 納品 に 対応 した モノ が 多い です から 、', ['どんどん と 少子 化 は 進む だろう', '現場 の 負担 は 「 現状 より 」 は 確実に 減る', '希望 通り の サイト に 一 度 飛び ます', '無駄な ストレス を 感じ ませ ん']],
 [2, 'c', 'b', '歩いて いる 子ども 達 を 見る と 、', ['まったく チェック して い なかった 事 を 悔やむ', '教室 に は まだ チラホラ と しか 人 が い ない', '恐れ や 将来 に 対する 不安 など 無い ように 見える', '１ 本 の 背 の 高い 木 が 見える だろう']],
 [2, 'c', 'a', '風 が 寒い ので 、', ['服装 に は 準備 が 必要だ', '色々な 種類 の カレー を 注文 し ます', '午後 から は 、 ゆっくり 体 を 休める こと に し ます', '自転車 に は 乗れ なく なる']],
 [2, 'a', 'd', '夕方 から は 友人 と ご飯 を 食べる ので', ['少し 気 が 楽だ', '血糖 値 の コントロール が でき ない', '一切 の 不安 無い', '睡眠 不足 は いくらか 解消 した ようだ']],
 [2, 'c', 'd', '会社 が 利益 を 出さ なければ', ['収入 が 激減 し ます', '親 が きちんと 謝罪 も お礼 も し なきゃ', '借金 は 返せ ませ ん', '収入 は 増え ない']],
 [2, 'd', 'c', '発生 した 責任 は 、 上司 に ある と 、', ['一 つ は 、 曖昧な 返事 しか 出来 ない', '県 外 の もの だった ため 盗ま れた もの か 確認 する', '当然 検事 総長 に も 重い 責任 が ある', '部下 は 上司 に ボール を 投げる']],
 [2, 'd', 'a', 'コマ を まわす と', ['目 が きょろきょろ 動く', '目 が キョロキョロ と 動き ます', '前輪 が 左右 に 動き ます', 'キョロちゃん が 起き上がる']],
 [2, 'b', 'c', '時間 通り に こ れ ない 人

In [27]:
import collections

cnt = collections.Counter(agreement)
print(cnt)
cnt_false = collections.Counter([line[0] for line in false])
print(cnt_false)
false_ratio = {'2':round(cnt_false[2]/cnt[2], 3), '3':round(cnt_false[3]/cnt[3], 3), '4':round(cnt_false[4]/cnt[4], 3)}
print(false_ratio)

Counter({3: 3673, 2: 3587, 4: 2968})
Counter({2: 568, 3: 441, 4: 292})
{'2': 0.158, '3': 0.12, '4': 0.098}
